<a href="https://colab.research.google.com/github/sherna90/analisis_algoritmos/blob/master/6.-tsp_or_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ssl
!pip install haversine
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ssl-1.16.tar.gz (33 kB)
  Using cached ssl-1.15.tar.gz (32 kB)
ERROR: Could not find a version that satisfies the requirement ssl (from versions: 1.15, 1.16)
ERROR: No matching distribution found for ssl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
import pandas as pd

In [70]:
distance=2
url='https://puntoslimpios.mma.gob.cl/api/points/geo?lat=-35.4232444&lng=-71.6484804&distance={}'.format(distance)

In [71]:
df=pd.read_json(url)

In [72]:
df

,distance,lat,status,lng,owner,type,address_type,manager,address_number,address_name,region,commune,materials
0,0.415525,-35.425537,open,-71.644859,CRISTORO,pv,,CRISTORO,,2 Norte,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[glass]
1,0.435133,-35.425625,open,-71.644669,CRISTORO,pv,,CRISTORO,,2 Norte,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[glass]
2,0.563316,-35.419284,open,-71.652357,Claudia Mocárquer,pv,Calle,Reciclaplast,1774,9 1/2 Oriente,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[plastic]
3,0.655458,-35.425101,open,-71.655346,Ciro Vergara Blanco,pv,Pasaje,Reciclaje Plasticos Vergara,,8 Oriente,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[plastic]
4,0.671793,-35.425675,open,-71.655268,guillermo toro,pv,,cristaleria toro,,8 Oriente,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[glass]
5,0.692395,-35.427981,open,-71.643520,Ciro Vergara Blanco,pv,,Reciclaje Plasticos Vergara,,1 Norte,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[plastic]
6,0.701577,-35.418307,open,-71.643660,guillermo toro,pv,,cristaleria toro,,14 Oriente,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[glass]
7,0.859046,-35.427758,open,-71.640786,guillermo toro,pv,,cristaleria toro,,18 Oriente,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[glass]
8,0.880842,-35.415556,open,-71.646137,Ilustre Municipalidad de Talca,pl,Calle,Ilustre Municipalidad de Talca,1957,Trece 1/2 Oriente,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}","[glass, paper, paperboard, cardboard_drink, pl..."
9,0.888974,-35.430895,open,-71.651328,Claudia Mocárquer,pv,,Reciclaplast,,12 Sur,"{'id': 7, 'abbreviation': 'ma', 'name': 'Maule'}","{'id': 116, 'name': 'Talca'}",[plastic]


In [73]:
coords=list()
for c in zip(df['lat'],df['lng']):
  coords.append(c)

In [74]:
import haversine as hs

def create_data_model(coords):
    """Stores the data for the problem."""
    data = {}
    dist_mat=[[0 for _ in  range(len(coords))] for _ in  range(len(coords))]
    locations={i:c for i,c in enumerate(coords)}
    for u in range(len(coords)):
      for v in range(len(coords)):
        dist_mat[u][v]=int(hs.haversine(locations[u],locations[v],unit='m'))
    data['distance_matrix'] = dist_mat
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data,locations

In [75]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} meters'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    route=[index]
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        route.append(index)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}meters\n'.format(route_distance)
    return route,route_distance


In [78]:
"""Simple Travelling Salesperson Problem (TSP) between cities."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

data,locations = create_data_model(coords)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


def distance_callback(from_index, to_index):
  """Returns the distance between the two nodes."""
  # Convert from routing variable Index to distance matrix NodeIndex.
  from_node = manager.IndexToNode(from_index)
  to_node = manager.IndexToNode(to_index)
  return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH)
search_parameters.time_limit.seconds = 30
search_parameters.log_search = True
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
print("Solver status: ", routing.status())
# Print solution on console.
if solution:
  route,route_value=print_solution(manager, routing, solution)

Solver status:  1
Objective: 18606 meters
Route for vehicle 0:
 0 -> 1 -> 7 -> 10 -> 21 -> 32 -> 30 -> 28 -> 29 -> 31 -> 33 -> 27 -> 19 -> 13 -> 14 -> 6 -> 8 -> 20 -> 41 -> 35 -> 38 -> 37 -> 17 -> 18 -> 43 -> 46 -> 42 -> 40 -> 15 -> 16 -> 12 -> 2 -> 3 -> 4 -> 9 -> 25 -> 26 -> 24 -> 36 -> 47 -> 44 -> 45 -> 34 -> 23 -> 39 -> 22 -> 11 -> 5 -> 0



In [69]:
import numpy as np

def make_cycle(current):
    value=0
    for c1,c2 in zip(current[:-1],current[1:]):
      value+=hs.haversine(tuple(c1),tuple(c2),unit='m')
    return value

current=np.random.permutation(coords).tolist()
current.append(current[0])
current_value=make_cycle(current)
print('random cycle : {} meters'.format(current_value))

random cycle : 5064.203137653731 meters


In [62]:
from itertools import permutations

perm=permutations(coords)

min_val=np.inf
for current in perm:
  current=list(current)
  current.append(current[0])
  current_value=make_cycle(current)
  min_val=min(min_val,current_value)
print('fuerza bruta {}'.format(min_val))

fuerza bruta 3134.209551020087


In [77]:
optimal_route=[coords[i] for i in route[:-1]]
optimal_route.append(optimal_route[0])
current_value=make_cycle(optimal_route)
print('optimal cycle : {} meters'.format(current_value))

optimal cycle : 18886.912200078415 meters
